# Triathlon Data - Preparing Dashboard

### context
This datas are giving a context to the relevance of our porject.
Triathlon is the combination of the three endurance sports: swim - bike - run and is one of the most growing discipline of the last 30 years.
First mentioned in the 20th of 19th century in France as "les trois sports" it got forgotten for decades.
Only in 1978 it was reinvented in Hawaii where the anual Ironman still can be named as the mother of this sport.

### aim
analogical to our dashboard for training athletes we want a interactive dashboard for the story of Triathlon from 1983 to 2022. 

### process
for making it interactive via parameters in tableau, we build some new columns: 
- year
- distance = full, half, olympic, sprint
- special_category = Professional Athletes, Para Athletes, Higher Weight Athletes, Relay
- new age classes = grouped 6 age ranges from below 19 to over 75

Espacially the categorical and the new age class assessment was complex due to many different person_event_group entries 

### results

-----------------------------------------------

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import io
import re
from scipy import stats
from scipy.stats import shapiro

import warnings
warnings.filterwarnings("ignore")

# 1. reading data from database

### SQAlchemy

In [2]:
from sqlalchemy import create_engine, types
from sqlalchemy import text # to be able to pass string
from sqlalchemy.exc import SQLAlchemyError
from dotenv import dotenv_values # to load the data from .env file

preparing connection

In [3]:
#load the data from .env file
config = dotenv_values()

# define variables for the login
pg_user = config['AZURE_USER'] 
pg_host = config['AZURE_HOST']
pg_port = config['AZURE_PORT']
pg_db = config['AZURE_DB']
pg_schema = config['AZURE_SCHEMA']
pg_pass = config['AZURE_PASS']

# build the URL
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

# create the engine
engine = create_engine(url, echo=False)
engine.url

# build the search path
my_schema = pg_schema 
with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {my_schema};'))

testing the connection

In [4]:
def test_db_connection(engine):
    try:
        connection = engine.connect() # including 'connection' as variable to close the connection
        print("Connection successful!")
        connection.close() # closing the connection
        return True
    except SQLAlchemyError as e:
        print(f"Connection failed: {e}")
        return False

# Test the connection
test_db_connection(engine)

Connection successful!


True

#### load DataFrame from Database

In [5]:
df_stats = pd.read_sql(sql=text('SELECT * FROM public.df_tri_stats_clean;'), con=engine)

df_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2958394 entries, 0 to 2958393
Data columns (total 12 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   event_link                object
 1   gender                    object
 2   person_link               object
 3   person_flag               object
 4   person_event_group        object
 5   person_event_swim_time    int64 
 6   person_event_cycle_time   int64 
 7   person_event_run_time     int64 
 8   person_event_t1_time      int64 
 9   person_event_t2_time      int64 
 10  person_event_finish_time  int64 
 11  person_country            object
dtypes: int64(6), object(6)
memory usage: 270.8+ MB


In [6]:
df_stats.describe()

,person_event_swim_time,person_event_cycle_time,person_event_run_time,person_event_t1_time,person_event_t2_time,person_event_finish_time
count,2.958394e+06,2.958394e+06,2.958394e+06,2.958394e+06,2.958394e+06,2.958394e+06
mean,2.929519e+12,1.346789e+13,1.014812e+13,3.439737e+11,3.355455e+11,2.747336e+13
std,2.173939e+12,7.336948e+12,6.396090e+12,6.375766e+11,1.330719e+12,1.439422e+13
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+10
25%,1.943000e+12,9.403000e+12,6.190000e+12,1.730000e+11,1.350000e+11,1.848500e+13
50%,2.530000e+12,1.138200e+13,8.315000e+12,2.990000e+11,2.300000e+11,2.301600e+13
75%,4.024000e+12,2.022800e+13,1.475800e+13,4.510000e+11,3.660000e+11,4.040600e+13
max,8.252400e+13,8.626300e+13,8.631700e+13,8.639900e+13,8.638900e+13,8.634800e+13


# 2. Preparing

## 'year' as new column

this will be the radial axis for the gender-plot in tableau

not all years are extractable by the last 4 - so check first if there is a '-'

In [7]:
# year out of the event_link string

import re

def extract_year(link):
    if '-' in link[-4:]:
        return link[-10:-6]  # if the date is in Format YYYY-MM-DD 
    match = re.search(r'/(\d{4})$', link)
    return match.group(1) if match else None

df_stats['year'] = df_stats['event_link'].apply(extract_year)

df_stats.head()

,event_link,gender,person_link,person_flag,person_event_group,person_event_swim_time,person_event_cycle_time,person_event_run_time,person_event_t1_time,person_event_t2_time,person_event_finish_time,person_country,year
0,/rus/result/ironman/ireland-cork/full/2019,M,/irl/profile/halliwell-mark,IRL,M45-49,0,28325000000000,19594000000000,0,1140000000000,49059000000000,Ireland,2019
1,/rus/result/ironman/ireland-cork/full/2019,F,/usa/profile/harris-polly,USA,F50-54,0,28733000000000,19329000000000,0,997000000000,49060000000000,United States,2019
2,/rus/result/ironman/ireland-cork/full/2019,M,/fra/profile/peugeot-rodolphe,FRA,M25-29,0,28625000000000,19810000000000,0,640000000000,49075000000000,France,2019
3,/rus/result/ironman/ireland-cork/full/2019,F,/gbr/profile/crawford-lisa,GBR,F50-54,0,27887000000000,20421000000000,0,771000000000,49079000000000,United Kingdom,2019
4,/rus/result/ironman/ireland-cork/full/2019,M,/usa/profile/brumit-aaron,USA,M45-49,0,25958000000000,21110000000000,0,2041000000000,49109000000000,United States,2019


In [9]:
# for using in tableau we need it as a dateformat,so that we bring the 1st january as month and day

df_stats['year'] = pd.to_datetime(df_stats['year'].astype(str) + '-01-01')

In [10]:
df_stats.head()

,event_link,gender,person_link,person_flag,person_event_group,person_event_swim_time,person_event_cycle_time,person_event_run_time,person_event_t1_time,person_event_t2_time,person_event_finish_time,person_country,year
0,/rus/result/ironman/ireland-cork/full/2019,M,/irl/profile/halliwell-mark,IRL,M45-49,0,28325000000000,19594000000000,0,1140000000000,49059000000000,Ireland,2019-01-01 01:00:00-01:00
1,/rus/result/ironman/ireland-cork/full/2019,F,/usa/profile/harris-polly,USA,F50-54,0,28733000000000,19329000000000,0,997000000000,49060000000000,United States,2019-01-01 01:00:00-01:00
2,/rus/result/ironman/ireland-cork/full/2019,M,/fra/profile/peugeot-rodolphe,FRA,M25-29,0,28625000000000,19810000000000,0,640000000000,49075000000000,France,2019-01-01 01:00:00-01:00
3,/rus/result/ironman/ireland-cork/full/2019,F,/gbr/profile/crawford-lisa,GBR,F50-54,0,27887000000000,20421000000000,0,771000000000,49079000000000,United Kingdom,2019-01-01 01:00:00-01:00
4,/rus/result/ironman/ireland-cork/full/2019,M,/usa/profile/brumit-aaron,USA,M45-49,0,25958000000000,21110000000000,0,2041000000000,49109000000000,United States,2019-01-01 01:00:00-01:00


## 'distance' as new column

this column will be an interactive parameter in our tableau dashborad

In [11]:
df_stats['event_link'].nunique()

6012

In [12]:
list(df_stats['event_link'].unique())

['/rus/result/ironman/ireland-cork/full/2019',
 '/rus/result/ironman/costa-rica/half/2019',
 '/rus/result/ironman/elsinore/half/2019',
 '/rus/result/ironman/mont-tremblant/half/2019',
 '/rus/result/ironman/luxembourg/half/2019',
 '/rus/result/ironman/costa-rica/olympic/2019',
 '/rus/result/nobrand/tatneft-альметьевский-триатлон/sprint/2019',
 '/rus/result/nobrand/pervenstvo-rossii-po-triatlonu/sprint/2019',
 '/rus/result/itu/asian-cup-gamagori/olympic/2019',
 '/rus/result/itu/national-championship-romania/sprint/2019',
 '/rus/result/itu/world-cup-antwerp/sprint/2019',
 '/rus/result/swissman/extreme-triathlon/full/2019',
 '/rus/result/ironman/mont-tremblant/olympic/2019',
 '/rus/result/3grom/ruza/olympic/2019',
 '/rus/result/itu/african-cup-larache/sprint/2019',
 '/rus/result/itu/junior-national-championship-russian-federation/sprint/2019',
 '/rus/result/itu/national-championship-czechia/half/2019',
 '/rus/result/itu/national-championship-poland/sprint/2019',
 '/rus/result/itu/national-

In [13]:

def extract_distance(link):
    parts = link.split('/')
    return parts[-2] if len(parts) > 1 else None

df_stats['distance'] = df_stats['event_link'].apply(extract_distance)

df_stats.head()


,event_link,gender,person_link,person_flag,person_event_group,person_event_swim_time,person_event_cycle_time,person_event_run_time,person_event_t1_time,person_event_t2_time,person_event_finish_time,person_country,year,distance
0,/rus/result/ironman/ireland-cork/full/2019,M,/irl/profile/halliwell-mark,IRL,M45-49,0,28325000000000,19594000000000,0,1140000000000,49059000000000,Ireland,2019-01-01 01:00:00-01:00,full
1,/rus/result/ironman/ireland-cork/full/2019,F,/usa/profile/harris-polly,USA,F50-54,0,28733000000000,19329000000000,0,997000000000,49060000000000,United States,2019-01-01 01:00:00-01:00,full
2,/rus/result/ironman/ireland-cork/full/2019,M,/fra/profile/peugeot-rodolphe,FRA,M25-29,0,28625000000000,19810000000000,0,640000000000,49075000000000,France,2019-01-01 01:00:00-01:00,full
3,/rus/result/ironman/ireland-cork/full/2019,F,/gbr/profile/crawford-lisa,GBR,F50-54,0,27887000000000,20421000000000,0,771000000000,49079000000000,United Kingdom,2019-01-01 01:00:00-01:00,full
4,/rus/result/ironman/ireland-cork/full/2019,M,/usa/profile/brumit-aaron,USA,M45-49,0,25958000000000,21110000000000,0,2041000000000,49109000000000,United States,2019-01-01 01:00:00-01:00,full


In [14]:
print(f'The interactive paramaters will be: {df_stats["distance"].unique()}')

The interactive paramaters will be: ['full' 'half' 'olympic' 'sprint' 'supersprint']


# prep: event groups 

In [15]:
df_stats['person_event_group'].nunique()

966

In [16]:
list(df_stats['person_event_group'].unique())


['M45-49',
 'F50-54',
 'M25-29',
 'F40-44',
 'M50-54',
 'M40-44',
 'F35-39',
 'M35-39',
 'M18-24',
 'M55-59',
 'M30-34',
 'F30-34',
 'F25-29',
 'F18-24',
 'M65-69',
 'M60-64',
 'F45-49',
 'F55-59',
 'F60-64',
 'M75-79',
 'MPRO',
 'FPRO',
 'M70-74',
 'F70-74',
 'F65-69',
 'M16-19',
 'F20-24',
 'MJUN16-19',
 'MJUN19-23',
 'FJUN16-19',
 'FJUN19-23',
 'M',
 'F',
 'MJUNIOR',
 'FJUNIOR',
 'MU15',
 'FU15',
 'MPTS5',
 'MPTS4',
 'MPTVI',
 'MPTWC',
 'FPTWC',
 'MPTS3',
 'FPTVI',
 'MPTS2',
 'FPTS2',
 'MPROEN',
 'MU23',
 'MFPROO',
 'FU23',
 'MFINALYOUTH',
 'MFINALBYOUTH',
 'FFINALYOUTH',
 'FFINALBYOUTH',
 'MSEMIFINAL1YOUTH',
 'MSEMIFINAL3YOUTH',
 'MSEMIFINAL2YOUTH',
 'FSEMIFINAL3YOUTH',
 'FSEMIFINAL1YOUTH',
 'FSEMIFINAL2YOUTH',
 'M60+',
 'MPC',
 'MBLUE',
 'MWHITE',
 'FWHITE',
 'FBLUE',
 'M30-39',
 'M50-59',
 'M40-49',
 'M24-29',
 'M18-23',
 'F30-39',
 'M9',
 'FPTS5',
 'FPTS4',
 'M80-84',
 'FPC',
 'F75-79',
 'M16-29',
 'F40-49',
 'F16-29',
 'M60-69',
 'M70+',
 'M20-24',
 'MPARA',
 'F16-19',
 'M18-29

#### problem: empty person_link


In [ ]:
# check for number of None in person_link

df_stats['person_link'].isna().sum()

There are > 66,000 participants over the years without person_link.

Before simply dropping them, I want to look for a pattern - maybe it's allways the same event over the years or it is just some nations. To then decide what to do with these data.


In [ ]:
# dataframe only with empty person_link

df_tri_noPerson=df_stats[df_stats['person_link'].isna()]

In [ ]:
# how many years

print(f"There are {df_tri_noPerson['year'].nunique()} years with unclear athletes")

That means, it is a phenomenon over the total period of time.

In [ ]:
# look for the number of events (once per year could be a sign for always the same)

print(f"There are {df_tri_noPerson['event_link'].nunique()} events with unclear athletes")

So it's not only one event.

In [ ]:
df_tri_noPerson.head(24)

As I'm heading for the age-group and maybe later for the person_country I check now how many are missing both of these information. In person_event_group 'M' and 'F' are not precise enough. Again, maybe that is due to a specific event (extreme-triathlon). I will check the coinstances.

In [ ]:
# first looking for country=None

df_tri_noPerson['person_country'].isna().sum()

So most of the doubtful rows don't give information for the country anyway. But let's check for the age-groups by the event_group. I assess M & F equaly to None, maybe there are more

In [ ]:
# check for the list of event_groups. 

list(df_tri_noPerson['person_event_group'].unique())

The following seem suspecious too: 'MNKNOWN'

So I would count 'None' + 'M' + 'F' + 'MNKNOWN'

In [ ]:
df_tri_drop = df_tri_noPerson[
    df_tri_noPerson['person_event_group'].isna() | 
    df_tri_noPerson['person_event_group'].isin(['M', 'F', 'MNKNOWN'])
]
df_tri_drop


Now I could just drop these 9,368 rows.

But I still would have 58.000 that I could not use for person_country. 

For deciding on that, I would like to calcuate the impact on the total data. If I drop them all I reduce the nunique() of person_event by just one. 


In [ ]:
print(f"The number of rows to drop would be {len(df_tri_noPerson)}, "
      f"\nthat is {(df_tri_noPerson['person_link'].nunique()) / (df_stats['person_link'].nunique()):.5%} of all athletes"
      f"\nbut {len(df_tri_noPerson) / len(df_stats):.2%} of all participations.")

I'm not sure if I will keep them in the end. For now I'll only drop those with unclear event_groups

In [ ]:
df_stats = df_stats.merge(df_tri_drop, how="left", indicator=True)
df_stats = df_stats[df_stats["_merge"] == "left_only"].drop(columns=["_merge"])

#### problem: unclear groups

There are 966 different groups - depending on event, the grouping varies. 

- Pro, Para, Relay and Clydesdale+Athena are not grouped by ages, but I will use them as a new column for special categories.


In [ ]:
# looking for all groups without clear numbers in it

def filter_entries_without_numbers(df, column_name):
    # filter for numbers ^over total string, \D+ only numbers, $ no number in the end
    return df[df[column_name].str.contains(r'^\D+$', na=False)]

# use function
filtered_df = filter_entries_without_numbers(df_stats, 'person_event_group')

# show result
list(filtered_df['person_event_group'].unique())


## categories  para / pro / A&C /relay

By manualy sorting the groups, I will build a new column with categories para, pro, A&C and relay - this will be a parameter in tableau

- para incl PTV (=blind), PTWC (): manualy sorted


- overweight A&C: Athena = female above 165 pounds (74,5kg) + Clydesdale =  male above 220 pounds (99,7kg): manualy sorted


- relay: 


- pro: search for pro in string




In [ ]:
# for preparing the categorical column - get the total of pro

df_stats[df_stats["person_event_group"].str.contains("pro", case=False, na=False)]["person_event_group"].unique().tolist()


In [ ]:
# build new column with categories


para_list = [
    'MPARA', 'MPARATHLETE', 'FPARATHLETE', 'MПАРААТЛЕТ', 'FПАРААТЛЕТ', 'PARATHLETE', 'TPARA', 
    'MPARAATHLETE', 'MPARATRIATHLETE', 'MPARATRI', 'FPARA', 'MAWADUPPEREXTREMITYBELOWELBOW', 
    'MAWADBLIND', 'MAWADOTHER', 'MAWADHANDCYCLE', 'MAWADUPPEREXTREMITY+ELBOW', 
    'MAWADLOWEREXTREMITYBELOWKNEE', 'FAWADLOWEREXTREMITYBELOWKNEE', 'FAWADHANDCYCLE', 
    'MAWADLOWEREXTREMITY+KNEE', 'FAWADBLIND', 'FAWADWHEELCHAIR', 'FAWADOTHER', 
    'MAWADDOUBLEAMPUTEEBELOWKNEE', 'FAWADUPPEREXTREMITYBELOWELBOW', 'MHANDCYCLE', 'MPTVI', 
    'FPTVI', 'MPTWC', 'FPTWC', 'FPARATRIAHLON40-49', 'MPTS1', 'MPTS2', 'MPTS3', 'MPTS4', 'MPTS5', 'MPTS5OPEN',
    'FPTS1', 'FPTS2', 'FPTS3', 'FPTS4', 'FPTS5', 'FPTS5OPEN',
    'MPTVI', 'FPTVI','MPTWC', 'FPTWC', 'MPTWCOPEN', 'FPTWCOPEN', 'PTVI', 'MPYB', 'FPYB',  'MPTWCOPEN', 'MPTVIOPEN','FPTWCOPEN', 'MPTS5OPEN',
    'MPTWCOPEN','MPTVIOPEN','MPTS4OPEN','MPTS2OPEN','MPTS3OPEN','FPTWCOPEN','MPT4OPEN','MPT5OPEN',
    'MPT1OPEN','MPT2OPEN','MPT3OPEN','FPT5OPEN','FPT1OPEN','FPT3OPEN', 'MPTTRI-1','MPTTRI-6A',
    'MPTTRI-2','FPTTRI-6B','FPTTRI-4','FPTTRI-1','FPTTRI-3','FPTTRI-5','FPTTRI-6A','FPTTRI-2','MPTTRI-6','FPTTRI-6', 'MPT1',
    'MPT5','MPT4','MPT2','FPT5','FPT4','MPT3','FPT1','MPTTRI-5','ПАРААТЛЕТ',  'MAWADWHEELCHAIR1','MAWADWHEELCHAIR2', 'MPTTRI-4',
    'MPTTRI-6B','MPTTRI-3', 'FAWADPC3','MAWADPC4','MAWADPC5','MAWADPC3','FAWADPC6','FAWADPC5','MAWADPC2','MAWADPC1',
    'FAWADPC2','MAWADPC6','FAWADPC4', 'FPC', 'MPC', 'FAWADPC1',  'PT5','PT4','PT3',  'FPT2','FPT3'
    ]

ac_list = ['MCLYDESDALE', 'FCLYDESDALE', 'FATHENAS', 'MCLYDESDALES', 'MCLY', 'FCLY', 'MCLY-U', 'MCLY-O','FATH', 'FATH-U', 'FATH-O']

relay_list = ['MRELAY', 'FRELAY', 'RELAY', 'MRELAYCOED', 'MIXED', 'M-RELAY', 'M-RLY', 'FX-RLY', 'F-RLY', 'MX-RLY', 
              'FRELAYCOED', 'MTEAM', 'FTEAM', 'FCOUPLES','MCOUPLES']

pro_list = ['MPRO',
    'FPRO','MPROEN','MFPROO','MSEMIFINAL3PRO','MSEMIFINAL2PRO','MSEMIFINAL1PRO','MPRODISTANCE','FPRODISTANCE','FSEMIFINAL3PRO',
    'FSEMIFINAL1PRO','FSEMIFINAL2PRO','MAGMPRO','FAGFPRO','MSEMIFINAL4PRO','FSEMIFINAL4PRO','MPROSEMIFINAL3','MPROSEMIFINAL2',
    'MPROSEMIFINAL1','FPROSEMIFINAL1','FPROSEMIFINAL2','MPRO30-34','MPRO40-44','MPRO25-29','MPRO35-39','FPRO30-34',
    'FPRO35-39','FPROILIPINOPRO','MPROANDU23','FPROANDU23','MQUALIFIER2PRO','MQUALIFIER1PRO','MSEMIFINAL2MPROEN',
    'MSEMIFINAL1MPROEN','MPRO1','MPROENSEMIFINAL2','MFPROOSEMIFINAL1','MQUALIFIER3PRO','MPROENSEMIFINAL3','MFPROOSEMIFINAL2',
    'MQUALIFIER4MPROEN','MQUALIFIER6MPROEN','MSEMIFINAL3MPROEN','MSEMIFINAL1FPROO','FFINALBPRO','MFINALCPRO',
    'MFINALBPRO','MFPROINALSTE1','MFPROINALSTE2','MFPROINALSTE3','FFPROINALSTE2','FFPROINALSTE1','FQUALIFIER1PRO',
    'FQUALIFIER2PRO','FFPROINALSTE3','MREPECHAGE2MPROEN','MQUALIFIER2MPROEN','MQUALIFIER4FPROO','MQUALIFIER5MPROEN',
    'MQUALIFIER1MPROEN','MREPECHAGE3MPROEN','MQUALIFIER2FPROO','MREPECHAGE1FPROO','MQUALIFIER3MPROEN','MQUALIFIER1FPROO',
    'MREPECHAGE4MPROEN','MREPECHAGE2FPROO','MREPECHAGE1MPROEN','MQUALIFIER3FPROO','MPROENREPECHAGE1STAGE1',
    'MPROENHEAT3STAGE1','MPROENREPECHAGE1STAGE2','MPROENHEAT3STAGE2','MPROENHEAT3','MFPROOREPECHAGE1STAGE1',
    'MFPROOHEAT1STAGE1','MFPROOREPECHAGE1STAGE2','MFPROOHEAT1STAGE2','MFPROOHEAT1','MPROENHEAT1STAGE1',
    'MPROENHEAT1STAGE2','MPROENHEAT1','MPROENREPECHAGE2STAGE1','MPROENREPECHAGE2STAGE2','MPROENHEAT2STAGE1',
    'MPROENHEAT2STAGE2','MPROENHEAT2','MFPROOHEAT2STAGE1','MFPROOHEAT2STAGE2','MFPROOHEAT2','FREPECHE1PRO','MREPECHE2PRO',
    'MREPECHE1PRO']


# function for assigning categories
def assign_category(person_event_group):
    if person_event_group in para_list:
        return 'para'
    elif person_event_group in ac_list:
        return 'a+c'
    elif person_event_group in relay_list:
        return 'relay'
    elif person_event_group in pro_list:
        return 'pro'
    else:
        return 'None'  # For the rows without

# generate new column
df_stats['special_category'] = df_stats['person_event_group'].apply(assign_category)

df_stats.head(25)


## translate age groups


In [ ]:
# looking for specific groups to see potential translation / assessment

df_search_groups = df_stats[df_stats['person_event_group'].isin(['MYOUTH',
'FYOUTH'])]

df_search_groups['event_link'].nunique()

### junior/youth as  <19

I group all Junior and Youth categories to one group '<19'


In [ ]:
df_young = df_stats[df_stats["person_event_group"].str.contains("ju", case=False, na=False)]
list(df_young['person_event_group'].unique())

In [ ]:
df_young = df_stats[df_stats["person_event_group"].str.contains("y", case=False, na=False)]
list(df_young['person_event_group'].unique())

In [ ]:
# translate category <19

df_stats['person_event_group'] = df_stats['person_event_group'].replace([
    'MJUNIOR', 'FJUNIOR', 'MSEMIFINAL1JUNIOR', 'MSEMIFINAL3JUNIOR', 'MSEMIFINAL2JUNIOR', 
    'FSEMIFINAL2JUNIOR', 'FSEMIFINAL1JUNIOR', 'MJUNIOREN18+19', 'MTIMETRIALQUALIFIERJUNIOR', 
    'FTIMETRIALQUALIFIERJUNIOR', 'MSEMIFINAL4JUNIOR', 'MFINALJUNIOR', 'FFINALJUNIOR', 
    'MJUNIORB', 'MJUNIORA', 'FJUNIORB', 'FJUNIORA', 'FSEMIFINAL3JUNIOR', 'MFINALBJUNIOR', 
    'FFINALBJUNIOR', 'MFINALYOUTH', 'MFINALBYOUTH', 'FFINALYOUTH', 'FFINALBYOUTH', 
    'MSEMIFINAL1YOUTH', 'MSEMIFINAL3YOUTH', 'MSEMIFINAL2YOUTH', 'FSEMIFINAL3YOUTH', 
    'FSEMIFINAL1YOUTH', 'FSEMIFINAL2YOUTH', 'MYOUTH', 'FYOUTH', 'MTIMETRIALQUALIFIERYOUTH', 
    'FTIMETRIALQUALIFIERYOUTH','MJUN19-23', 'MJUN16-19', 'FJUN16-19', 'FJUN19-23', 'MJUN', 'FJUN', 'MJU', 'MJUN', 'YM18','YW18',
    'YW16','YM16', 'M19UND','F19UND'
], '<19')


### veterans of Deltebre

Now we have to find translations for the not intuitive ones:

- MVETERANS1, MVETERANS2, MVETERANS3, FVETERANS1, FVETERANS2

In [ ]:
# looking for the MVETERANS
# which event uses these categories to then research for the meaning

df_veterans = df_stats[df_stats["person_event_group"].str.contains("veterans", case=False, na=False)]
list(df_veterans['event_link'].unique())

So they MVETERANS come from the TriTour Event in Deltebre in Catalonia.
A websearch could calrify :
- Youth = 16-17 years
- Junior = 18-19 years
- U23 = 20-23 years
- Senior = 24-39 years
- Veterans 1 = 40-49 years
- Veterans 2 = 50-59 years
- Veterans 3 = above 60 years

In [ ]:
# replace the entrys

df_stats['person_event_group'] = df_stats['person_event_group'].replace({
    'MVETERANS1': '40-49',
    'FVETERANS1': '40-49',
    'MVETERANS2': '50-59',
    'FVETERANS2': '50-59',
    'MVETERANS3': '>60'
})

### senior

In [ ]:
df_senior = df_stats[df_stats["person_event_group"].str.contains("senior", case=False, na=False)]
list(df_senior['person_event_group'].unique())

So there are some categories for seniors - I'll try to get information on of the numbered may belong to one particular event

In [ ]:
senior_groups = [
    'MSENIOR4', 'MSENIOR2', 'MSENIOR3', 'MSENIOR1',
    'FSENIOR4', 'FSENIOR3', 'FSENIOR2', 'FSENIOR1'
]

df_seniors = df_stats[df_stats['person_event_group'].isin(senior_groups)]
df_seniors['event_link'].unique()

Web research for Rimini Triathlon:

- Under23	20 - 23
- Senior 1	20 - 24
- Senior 2	25 - 29
- Senior 3	30 - 34
- Senior 4	35 - 39
- Master 1	40 - 44
- Master 2	45 - 49
- Master 3	50 - 54
- Master 4	55 - 59
- Master 5	60 - 64
- Master 6	65 - 69
- Master 7	70 - 74
- Master 8	75

In [ ]:
# replace the entrys

df_stats['person_event_group'] = df_stats['person_event_group'].replace({
    'MSENIOR1': '20-24', 
    'FSENIOR1': '20-24',
    'MSENIOR2': '25-29', 
    'FSENIOR2': '25-29',
    'MSENIOR3': '30-34', 
    'FSENIOR3': '30-34',
    'MSENIOR4': '35-39', 
    'FSENIOR4': '35-39'
})


but still there are 'senior'

In [ ]:
df_senior = df_stats[df_stats["person_event_group"].str.contains("senior", case=False, na=False)]
df_senior['event_link'].unique()

we know Delterbe already and I did a research for the politie triathlon in the Netherlands. This event gives some problems anyway, let's have a closer look on it

In [ ]:
df_politie = df_stats[df_stats["event_link"].str.contains("politie", case=False, na=False)]
df_senior['person_link'].nunique()

So I will use the translation of Delterbe and drop the politie event with 112 athletes

In [ ]:
# drop politie

df_stats = df_stats[~df_stats["event_link"].str.contains("politie", case=False, na=False)]


In [ ]:
# translate senior according to Delterbe

df_stats['person_event_group'] = df_stats['person_event_group'].replace({
    'MSENIOR': '24-39', 
    'FSENIOR': '24-39'
})

### masters

In [ ]:
# looking for the MASTERS
# which event uses these categories to then research for the meaning

df_master = df_stats[df_stats["person_event_group"].str.contains("master", case=False, na=False)]

print("The events are:")
list(df_master['event_link'].unique())


In [ ]:
# group by event_link than aggregate the list of category names and another list with one entry the number of individual athletes
grouped = df_master.groupby('event_link').agg(
    unique_groups=('person_event_group', 'unique'),
    unique_person_links=('person_link', 'nunique')
)

# iterate through the rows 
print("The events with their corresponding group names and unique person counts are: \n")
for event, row in grouped.iterrows():
    print(f"Event: {event}\nGroups: {', '.join(row['unique_groups'])}\nUnique person links: {row['unique_person_links']}\n")



We already know for Rimini:

- Master 1	40 - 44
- Master 2	45 - 49
- Master 3	50 - 54
- Master 4	55 - 59
- Master 5	60 - 64
- Master 6	65 - 69
- Master 7	70 - 74
- Master 8	75

In [ ]:
# replace the entrys

df_stats['person_event_group'] = df_stats['person_event_group'].replace({
    'FMASTERS1': '40-44',
    'MMASTERS1': '40-44', 
    'FMASTERS2': '45-49',
    'MMASTERS2': '45-49',  
    'FMASTERS3': '50-54',
    'MMASTERS3': '50-54',
    'MMASTERS4': '55-59', 
    'MMASTERS5': '60-64',
    'MMASTERS6': '65-69'
})


as I don't get information for the older two events but the total is only 120 athletes, I will drop these rows.

In [ ]:
# list of entries to drop
exclude_groups = ['FMASTERSV1', 'MASTERSV1', 
                  'FMASTERSV2', 'MASTERSV2', 
                  'FMASTERSV3', 'MASTERSV3', 
                  'FMASTERSV4', 'MASTERSV4', 
                  'FMASTERSV5', 'MASTERSV5']

# use list to drop
df_stats = df_stats[~df_stats['person_event_group'].isin(exclude_groups)]


### open

In [ ]:
# looking for the groups - f.e. everything regarding 'open' (for para I had to check manualy)

df_open = df_stats[df_stats["person_event_group"].str.contains("open", case=False, na=False)]
list(df_open['person_event_group'].unique())

Most of the open categories can either be traducted to an age or are already in the para-group. Only the MOPEN and FOPEN don't give any information at the moment. So let's look for then

In [ ]:
df_open = df_stats[df_stats["person_event_group"].str.contains("MOPEN|FOPEN", case=False, na=False)]

print(df_open['event_link'].unique())

print(f"\nThe number of athletes all over these events is {df_open['person_link'].nunique()}") 


I will drop these athletes due to being 0.0001% of the total

In [ ]:
# drop MOPEN and FOPEN

df_stats = df_stats[~df_stats["person_event_group"].str.contains("MOPEN|FOPEN", case=False, na=False)]

### what's left

In [ ]:
# repeat looking for unclear groups - only include the rows where 'special_category' is None


def filter_entries_without_numbers(df, column_name):
    # build filter where special_category is None
    df = df[df['special_category']== 'None']
    return df[df[column_name].str.contains(r'^\D+$', na=False)]

# use function
filtered_df = filter_entries_without_numbers(df_stats, 'person_event_group')

# show result
print(f"There are {filtered_df['person_event_group'].nunique()} unclear categories for "
      f"{filtered_df['person_link'].nunique()} individual athletes")
list(filtered_df['person_event_group'].unique())



Exemplary I check these entries

- MWHITE, MBLUE, MBLACK, MYELLOW
- FXTRI, MXTRI
- FTBC, MTBC

In [ ]:
# looking for the  MWHITE, MBLUE, MBLACK, MYELLOW
# which event uses these categories to then research for the meaning

df_colour= df_stats[df_stats["person_event_group"].str.contains("MWHITE|MBLUE|MBLACK|MYELLOW", case=False, na=False)]

print(f"There are {df_colour['person_link'].count()} participant in {df_colour['event_link'].nunique()} events.\nThe list of events is \n{df_colour['event_link'].unique()}")


In [ ]:
# checking for FXTRI|MXTRI|FTBC|MTBC

df_colour= df_stats[df_stats["person_event_group"].str.contains("FXTRI|MXTRI|FTBC|MTBC", case=False, na=False)]

print(f"There are {df_colour['person_link'].count()} participant in {df_colour['event_link'].nunique()} events.\nThe list of events is \n{df_colour['event_link'].unique()}")

these groups refer to different courses and don't give any hint on agegroup. As it is about 824 participants, I drop these rows.

In [ ]:
# droping FXTRI|MXTRI|FTBC|MTBC & MWHITE, MBLUE, MBLACK, MYELLOW

# call the entries to be droped
exclude_values = ['FXTRI', 'MXTRI', 'FTBC', 'MTBC', 'MWHITE', 'MBLUE', 'MBLACK', 'MYELLOW']

# keep those that are not (~) including the excluded values
df_stats = df_stats[~df_stats['person_event_group'].isin(exclude_values)]


I'll check the amount of all the unclear groups to decide if to drop them all

In [ ]:
print(f"The number of athletes to drop would be {filtered_df['person_link'].nunique()}, "
      f"\nwhich is {filtered_df['person_link'].nunique() / df_stats['person_link'].nunique():.2%} of all athletes.")


Together with the 2.25% of rows without person_link, I had to drop about 5% of the data.
But as I need the person_link for comparing if an athlete apears twice or more on an event, I obligatorily need the information out of this column.
I will drop the empty person_link

In [ ]:
# drop all rows without person_link

df_stats = df_stats.dropna(subset=['person_link'])


## athlets per event

Some of these groups suggest, that an athlete may has changed his group from one to another during an event, f.e. from qualifier to finalist.

To make sure that each athlete isn't counted more then once per event, I will check for duplicates depending on event_link and person_link.

In [ ]:
# check for duplicates again - maybe there was something wrong

duplicates_s = df_stats[df_stats.duplicated(keep=False)]

print('Number of duplicats: ', len(duplicates_s))

In [ ]:
# drop duplicates again

df_stats = df_stats.drop_duplicates(keep = 'first', ignore_index = True, inplace=False)

In [ ]:
# looking for double athletes on one event, due to changing the group

df_double_event = df_stats[df_stats.duplicated(subset=['person_link', 'event_link'], keep=False)]

# drop doubled combinations
df_double_event = df_double_event.drop_duplicates(subset=['person_link', 'event_link', 'person_event_group'])

# group to see wich person_event_link per person_link
duplicates_grouped = df_double_event.groupby(['event_link', 'person_link'])['person_event_group'].apply(lambda x: list(set(x))).reset_index()


print(f"There are {len(duplicates_grouped)} entries on the list"
      f"\nand {duplicates_grouped['person_link'].nunique()} individual athletes")
duplicates_grouped.head()


The numbers should be the same - as I group by the combination of one event and one athlete the total length and the number of indovidual athletes should be the same.

I'll try to find the error

In [ ]:
# wich person_link come up more then once

duplicates_grouped['person_link'].value_counts().head(20)


In [ ]:
# looking for the example of "/aus/profile/jeffcoat-emma" to get more insights

df_double_event[df_double_event['person_link'] == "/aus/profile/jeffcoat-emma"]


In [ ]:
# how often on wich event

df_double_event[df_double_event['person_link'] == "/aus/profile/jeffcoat-emma"]['event_link'].value_counts()


So the reason for the difference between length and nunique() ist due to one athlete being doublede on several events. 

But is it sure that each athlete is realy at least twice on an event? I prefer to check that

In [ ]:
# count athletes person_link per event_link
counts = df_double_event.groupby(['event_link', 'person_link']).size()

# all group combis (event-person) that are at least twice
valid_persons = counts[counts >= 2].reset_index()[['event_link', 'person_link']]

# filter only the valid person
df_double_event = df_double_event.merge(valid_persons, on=['event_link', 'person_link'], how='inner')

duplicates_grouped = df_double_event.groupby(['event_link', 'person_link'])['person_event_group'].apply(list).reset_index()

print(f"The total number of rows is {len(df_double_event)} and can be grouped to a \n"
      f"list with {len(duplicates_grouped)} entries of {duplicates_grouped['event_link'].nunique()} events"
      f"\nfrom {duplicates_grouped['person_link'].nunique()} individual athletes"
      # check if each athlete is there at least two times
      f"\neach athlete: {df_double_event.groupby(['event_link', 'person_link']).size().min()} times per event")


#### reasons for duplications

now we can bring together the rows of one athlete per event, depending on the reason for duplicates:

1. changing during the race (from qualifier to finalist / for youth & junior already solved, so that will happen during assessment to new overall age groups)

2. including the special_category information to the age_group information (pro)

3. two age-groups due to mistake (F & M) or due to in between two agegroups : search for the same athlete in other events, compare with year what age-group would be consequence

### 1. drop finalist

During a race the athlete may have become a finalist. So all groups including *finalist get dropped

In [ ]:
# drop all rows that include the word *final (semifinalist, finalist..) - keep age-related group or the first final row

# function that checks if there would remain a row without 'final' 
def resolve_final_entries(group):
    # is there 'final' in the group 
    final_entries = group['person_event_group'].str.contains("final", case=False, na=False)
    # is there another entry without final
    if final_entries.any() and (not final_entries.all()):
        # Keep only rows that do not contain 'final'
        return group[~final_entries]
    # If all entries are 'final', keep only the first one
    if final_entries.all():
        return group.head(1)
    # if no final, keep the group untouched
    return group

# use the filter function (apply as we want to drop rows)
df_double_event = df_double_event.groupby(['event_link', 'person_link']).apply(resolve_final_entries).reset_index(drop=True)

# compare to before
duplicates_grouped = df_double_event.groupby(['event_link', 'person_link'])['person_event_group'].apply(list).reset_index()

print(f"The total number of rows is {len(df_double_event)} and can be grouped to a \n"
      f"list with {len(duplicates_grouped)} entries of {duplicates_grouped['event_link'].nunique()} events"
      f"\nfrom {duplicates_grouped['person_link'].nunique()} individual athletes"
      # check if each athlete is there at least two times
      f"\neach athlete: {df_double_event.groupby(['event_link', 'person_link']).size().min()} times per event")

### 2. special category

if the athlete defined by 'person_link' is registered twice on an event, compare the 'special_category' and integrate
this information to all the rows of this athlete in this event.
After that dropping the rows of this athlete in this event where person_event_group is out of the list of special categories


In [ ]:
# bring the categorical information out of 'special_category' to all row per group (event+person)


def resolve_special_category(group):
    # build a list of all values that aren't 'None'
    unique_values = [val for val in group.unique() if not pd.isna(val) and val != "None"] 
    
    # in case of more then one special_category, mark it seperatly
    if len(unique_values) > 1:
        group["is_multiple_categories"] = True  
        return group  # skip replacement
    
    # in case of 'None' + one unique other value: replace
    if "None" in group.values and unique_values:
        return group.replace("None", unique_values[0])  # unique_value instead of None
    
    return group  # if only None, nothing happens

# by using .transform() it's used in each group (event_link and person_link) on each row
df_double_event["special_category"] = df_double_event.groupby(["event_link", "person_link"])["special_category"].transform(resolve_special_category)

# check if there are any cases of more then one special_category
df_double_event["is_multiple_categories"] = df_double_event.groupby(["event_link", "person_link"])["special_category"].transform(
    lambda x: len(x.unique()) > 1
)

df_multiple_categories = df_double_event[df_double_event["is_multiple_categories"]]

df_multiple_categories

In [ ]:
# compare to before
duplicates_grouped = df_double_event.groupby(['event_link', 'person_link'])['person_event_group'].apply(list).reset_index()

print(f"The total number of rows is {len(df_double_event)} and can be grouped to a \n"
      f"list with {len(duplicates_grouped)} entries of {duplicates_grouped['event_link'].nunique()} events"
      f"\nfrom {duplicates_grouped['person_link'].nunique()} individual athletes"
      # check if each athlete is there at least two times
      f"\neach athlete: {df_double_event.groupby(['event_link', 'person_link']).size().min()} times per event")


Now we can drop the person_event_group that gave the additional information for the special_category. 
For curiosity I first have a look, what categorical information without age is there.


In [ ]:
# person_event_groups in here without numbers

def filter_entries_without_numbers(df, column_name):
    # filter for numbers ^over total string, \D+ only numbers, $ no number in the end
    return df[df[column_name].str.contains(r'^\D+$', na=False)]

# use function
filtered_df = filter_entries_without_numbers(df_double_event, 'person_event_group')

# show result
list(filtered_df['person_event_group'].unique())

As I can't be sure that the special categories are doubled with another age-group category I will make sure not to loose athletes by dropping the categorical rows.

(I will also have to think of that when building our new age-groups, that there will be a big group without information about age.)

Nevertheless, I use the total special_list to drop the rows that we used for the information that are now transferred to the age-group relative row.

In [ ]:
# list of special categories

special_list = [
    # para
    'MPARA', 'MPARATHLETE', 'FPARATHLETE', 'MПАРААТЛЕТ', 'FПАРААТЛЕТ', 'PARATHLETE', 'TPARA', 
    'MPARAATHLETE', 'MPARATRIATHLETE', 'MPARATRI', 'FPARA', 'MAWADUPPEREXTREMITYBELOWELBOW', 
    'MAWADBLIND', 'MAWADOTHER', 'MAWADHANDCYCLE', 'MAWADUPPEREXTREMITY+ELBOW', 
    'MAWADLOWEREXTREMITYBELOWKNEE', 'FAWADLOWEREXTREMITYBELOWKNEE', 'FAWADHANDCYCLE', 
    'MAWADLOWEREXTREMITY+KNEE', 'FAWADBLIND', 'FAWADWHEELCHAIR', 'FAWADOTHER', 
    'MAWADDOUBLEAMPUTEEBELOWKNEE', 'FAWADUPPEREXTREMITYBELOWELBOW', 'MHANDCYCLE', 'MPTVI', 
    'FPTVI', 'MPTWC', 'FPTWC', 'FPARATRIAHLON40-49', 'MPTS1', 'MPTS2', 'MPTS3', 'MPTS4', 'MPTS5', 'MPTS5OPEN',
    'FPTS1', 'FPTS2', 'FPTS3', 'FPTS4', 'FPTS5', 'FPTS5OPEN',
    'MPTVI', 'FPTVI','MPTWC', 'FPTWC', 'MPTWCOPEN', 'FPTWCOPEN', 'PTVI', 'MPYB', 'FPYB',  'MPTWCOPEN', 'MPTVIOPEN','FPTWCOPEN', 'MPTS5OPEN',
    'MPTWCOPEN','MPTVIOPEN','MPTS4OPEN','MPTS2OPEN','MPTS3OPEN','FPTWCOPEN','MPT4OPEN','MPT5OPEN',
    'MPT1OPEN','MPT2OPEN','MPT3OPEN','FPT5OPEN','FPT1OPEN','FPT3OPEN', 'MPTTRI-1','MPTTRI-6A',
    'MPTTRI-2','FPTTRI-6B','FPTTRI-4','FPTTRI-1','FPTTRI-3','FPTTRI-5','FPTTRI-6A','FPTTRI-2','MPTTRI-6','FPTTRI-6', 'MPT1',
    'MPT5','MPT4','MPT2','FPT5','FPT4','MPT3','FPT1','MPTTRI-5','ПАРААТЛЕТ',  'MAWADWHEELCHAIR1','MAWADWHEELCHAIR2', 'MPTTRI-4',
    'MPTTRI-6B','MPTTRI-3', 'FAWADPC3','MAWADPC4','MAWADPC5','MAWADPC3','FAWADPC6','FAWADPC5','MAWADPC2','MAWADPC1',
    'FAWADPC2','MAWADPC6','FAWADPC4', 'FPC', 'MPC', 'FAWADPC1',  'PT5','PT4','PT3',  'FPT2','FPT3',
    # a&c
    'MCLYDESDALE', 'FCLYDESDALE', 'FATHENAS', 'MCLYDESDALES', 'MCLY', 'FCLY', 'MCLY-U', 'MCLY-O','FATH', 'FATH-U', 'FATH-O',
    # relay
    'MRELAY', 'FRELAY', 'RELAY', 'MRELAYCOED', 'MIXED', 'M-RELAY', 'M-RLY', 'FX-RLY', 'F-RLY', 'MX-RLY', 
    'FRELAYCOED', 'MTEAM', 'FTEAM', 'FCOUPLES','MCOUPLES',
    # pro
    'MPRO',
    'FPRO','MPROEN','MFPROO','MSEMIFINAL3PRO','MSEMIFINAL2PRO','MSEMIFINAL1PRO','MPRODISTANCE','FPRODISTANCE','FSEMIFINAL3PRO',
    'FSEMIFINAL1PRO','FSEMIFINAL2PRO','MAGMPRO','FAGFPRO','MSEMIFINAL4PRO','FSEMIFINAL4PRO','MPROSEMIFINAL3','MPROSEMIFINAL2',
    'MPROSEMIFINAL1','FPROSEMIFINAL1','FPROSEMIFINAL2','MPRO30-34','MPRO40-44','MPRO25-29','MPRO35-39','FPRO30-34',
    'FPRO35-39','FPROILIPINOPRO','MPROANDU23','FPROANDU23','MQUALIFIER2PRO','MQUALIFIER1PRO','MSEMIFINAL2MPROEN',
    'MSEMIFINAL1MPROEN','MPRO1','MPROENSEMIFINAL2','MFPROOSEMIFINAL1','MQUALIFIER3PRO','MPROENSEMIFINAL3','MFPROOSEMIFINAL2',
    'MQUALIFIER4MPROEN','MQUALIFIER6MPROEN','MSEMIFINAL3MPROEN','MSEMIFINAL1FPROO','FFINALBPRO','MFINALCPRO',
    'MFINALBPRO','MFPROINALSTE1','MFPROINALSTE2','MFPROINALSTE3','FFPROINALSTE2','FFPROINALSTE1','FQUALIFIER1PRO',
    'FQUALIFIER2PRO','FFPROINALSTE3','MREPECHAGE2MPROEN','MQUALIFIER2MPROEN','MQUALIFIER4FPROO','MQUALIFIER5MPROEN',
    'MQUALIFIER1MPROEN','MREPECHAGE3MPROEN','MQUALIFIER2FPROO','MREPECHAGE1FPROO','MQUALIFIER3MPROEN','MQUALIFIER1FPROO',
    'MREPECHAGE4MPROEN','MREPECHAGE2FPROO','MREPECHAGE1MPROEN','MQUALIFIER3FPROO','MPROENREPECHAGE1STAGE1',
    'MPROENHEAT3STAGE1','MPROENREPECHAGE1STAGE2','MPROENHEAT3STAGE2','MPROENHEAT3','MFPROOREPECHAGE1STAGE1',
    'MFPROOHEAT1STAGE1','MFPROOREPECHAGE1STAGE2','MFPROOHEAT1STAGE2','MFPROOHEAT1','MPROENHEAT1STAGE1',
    'MPROENHEAT1STAGE2','MPROENHEAT1','MPROENREPECHAGE2STAGE1','MPROENREPECHAGE2STAGE2','MPROENHEAT2STAGE1',
    'MPROENHEAT2STAGE2','MPROENHEAT2','MFPROOHEAT2STAGE1','MFPROOHEAT2STAGE2','MFPROOHEAT2','FREPECHE1PRO','MREPECHE2PRO',
    'MREPECHE1PRO'
    ]



But to make sure not to drop an athlete totaly if he/she has no other 'person_event_group' we have to include that to our code and validate for dropping first.

In [ ]:
## drop the rows if information could be transferred

df_double_special = df_double_event

# mark the rows tht include entries from special_list
df_double_special['is_special'] = df_double_special['person_event_group'].isin(special_list)

# make a subset with those groups that have more then one row
multiple_rows_groups = df_double_special.groupby(['event_link', 'person_link']).filter(lambda x: len(x) > 1)

# if there is more then one, prefer to keep the one without special_entry but keep at least one 
def resolve_special_cateory(group):
    if group['is_special'].any(): # check for any entrys of special_list
        group_non_special = group[~group['is_special']]  # take off rows with special_list entry
        if len(group_non_special) > 0:  # if there is one without, keep that
            group = group_non_special
        else:  # if all rows are in special_list, keep the first
            group = group.head(1)
    return group

# use filter on each group (apply as we want to change the structure)
df_filtered = multiple_rows_groups.groupby(['event_link', 'person_link']).apply(resolve_special_cateory)

# prepare the single rows that were excluded in multiple_rows_group
df_single_row_groups = df_double_special[~df_double_special['person_link'].isin(multiple_rows_groups['person_link'])]

# bring both groups together again
df_double_event = pd.concat([df_filtered, df_single_row_groups])

# due to concat reset index
df_double_event = df_double_event.reset_index(drop=True)


In [ ]:
# compare to before
duplicates_grouped = df_double_event.groupby(['event_link', 'person_link'])['person_event_group'].apply(list).reset_index()

print(f"The total number of rows is {len(df_double_event)} and can be grouped to a \n"
      f"list with {len(duplicates_grouped)} entries of {duplicates_grouped['event_link'].nunique()} events"
      f"\nfrom {duplicates_grouped['person_link'].nunique()} individual athletes"
      # check if each athlete is there at least two times
      f"\neach athlete: {df_double_event.groupby(['event_link', 'person_link']).size().min()} times per event")

duplicates_grouped.head(10)

### 3.a wrong gender

In [ ]:
# function to check if gender and person_event_group are alliigned

def resolve_gender_mismatch(group):
    # extract the letters M or F from person_event_group
    group['gender_letter'] = group['person_event_group'].str.extract(r'([MF])', expand=False)
    
    # group of rows where gender and person_event_group are matching
    gender_correct = group['gender_letter'] == group['gender']
    
    if gender_correct.any() and len(group) > 1:
        # if there is a dismatch only keep the matching
        return group[gender_correct] if gender_correct.sum() > 0 else group.head(1)
    else:
        # if only one row or all are matching don't change anything
        return group

# use function on groups
df_double_event = df_double_event.groupby(['event_link', 'person_link']).apply(resolve_gender_mismatch)

# Optional: Index zurücksetzen
df_double_event = df_double_event.reset_index(drop=True)


In [ ]:
# compare to before
duplicates_grouped = df_double_event.groupby(['event_link', 'person_link'])['person_event_group'].apply(list).reset_index()

print(f"The total number of rows is {len(df_double_event)} and can be grouped to a \n"
      f"list with {len(duplicates_grouped)} entries of {duplicates_grouped['event_link'].nunique()} events"
      f"\nfrom {duplicates_grouped['person_link'].nunique()} individual athletes"
      # check if each athlete is there at least two times
      f"\neach athlete: {df_double_event.groupby(['event_link', 'person_link']).size().min()} times per event")

duplicates_grouped.head(10)

### integrate info to df_stats

Allthough there are still doubled groups I integrate the information I got so far to the main Dataframe df_stats because I will head for the age-groups now and at least some of the doublage will be solved autmatically

In [ ]:
# find all rows that has to be dropped from df_stats

df_double_drop = df_stats[df_stats.duplicated(subset=['person_link', 'event_link'], keep=False)]
print(f"Doubled groups event+person in df_stats: {len(df_double_drop)}")

In [ ]:
print(f"after dropping in df_double_event rows: {len(df_double_event)}")

In [ ]:
# find all rows that has to be dropped from df_stats

df_double_drop = df_stats[df_stats.duplicated(subset=['person_link', 'event_link'], keep=False)]

# Schritt 2: Lösche die Zeilen aus df_stats, die in df_double_drop enthalten sind
df_stats_filtered = df_stats.merge(df_double_drop, how='left', indicator=True)
df_stats_filtered = df_stats_filtered[df_stats_filtered['_merge'] == 'left_only'].drop(columns=['_merge'])


# Schritt 3: Füge df_double_event zu df_stats_filtered hinzu
df_stats_updated = pd.concat([df_stats_filtered, df_double_event], ignore_index=True)

# Kontrolle: Überprüfe die Anzahl der Zeilen vor und nach der Änderung
print(f"Original df_stats rows: {len(df_stats)}")
print(f"Filtered df_stats rows: {len(df_stats_filtered)}")
print(f"Updated df_stats rows: {len(df_stats_updated)}")


In [ ]:
df_stats = df_stats_updated

# prep: age classes

For building new age groups for visualisation, I first have to decide what grouping would be reasonable.

First aproach would be
- <20
- 20-34
- 35-44
- 45-59
- 60-74
- ->75

1. I'll build a new column age_group to extract the numeric information out of person_event_group.
2. I'll build another new column new_age_class that then will group by our own system

### 1. age_group: new column

get only the age as a new column to compare and in the end drop the person_event_group

In [ ]:

# function to extract age_group
def extract_age_group(person_event_group):
    if pd.isna(person_event_group):
        return None
    
    # 1. pattern "number number - number number" (f.e. "20-24")
    match = re.search(r'(\d{2}-\d{2})', person_event_group)
    if match:
        return match.group(1)
    
    # 2. pattern "number number +" (f.e. "20+")
    match = re.search(r'(\d{2}\+)$', person_event_group)
    if match:
        return match.group(1)
    
     # 3. pattern "number number" (f.e. "17", "23")
    match = re.search(r'(\d{2})$', person_event_group)
    if match:
        return match.group(1)
    
     # 4. pattern for  "U number number" (f.e. "U23") or "MU number number" (z.B. "MU17")
    match = re.search(r'(U\d{2}|MU\d{2})$', person_event_group)
    if match:
        return match.group(1)
    
    # 5. pattern for "letter number number" (f.e. M17, F25) exept letter "U"
    match = re.search(r'([MF])(\d{2})$', person_event_group)
    if match:
        if match.group(1) == 'U':
            return match.group(0)  # f.e. "U23"
        else:
            return match.group(2)  # just number f.e. "17" or "25"
    
    # 6. pattern "> number number" f.e. '<19' or '>60'
    match = re.search(r'(<\d+|\d+>)$', person_event_group)
    if match:
        return match.group(1)

    return None

# new column age_group
df_stats['age_group'] = df_stats['person_event_group'].apply(extract_age_group)

# print result
df_stats[['person_event_group', 'special_category', 'age_group']].head(15)



In [ ]:
# how many empty age_group

print(f"There are {df_stats['age_group'].isna().sum():,} rows without age-group"
      f"\nand {df_stats[df_stats['age_group'].isna()]['person_link'].nunique():,} individual athletes " 
      f"out of {df_stats['person_link'].nunique():,} in total.")

Do these 52,000 have entries in special_category?

In [ ]:
# list of all athletes without 'age_group'
athletes_without_age_group = df_stats[df_stats['age_group'].isna()]['person_link'].unique()

# check for entries in special_category
num_without_special_category = df_stats[
   df_stats['person_link'].isin(athletes_without_age_group) & 
    (df_stats['special_category'].isna() | (df_stats['special_category'] == "None"))
]['person_link'].nunique()

# total of athletes without age_group
total_athletes_without_age_group = len(athletes_without_age_group)

# print result
if num_without_special_category == total_athletes_without_age_group:
    print("ALl athletes have an entry in special_category.")
else:
    print(f"{num_without_special_category:,} of the {total_athletes_without_age_group:,} Athletes don't have informations in special_category.\n")

# Subset DataFrame of Athletes without 'age_group' and 'special_category'
df_missing_both = df_stats[df_stats['age_group'].isna() & (df_stats['special_category'].isna() | (df_stats['special_category'] == "None"))]

df_missing_both.head()


What person_event_group entries are there

In [ ]:
print(f"The number of unclear groups is {df_missing_both['person_event_group'].nunique()}")
list(df_missing_both['person_event_group'].unique())

There are Nones - let's check them first

In [ ]:
none_rows_count = df_missing_both['person_event_group'].isna().sum()
unique_athletes_count = df_missing_both[df_missing_both['person_event_group'].isna()]['person_link'].nunique()

print(f"The number of 'None' rows is {none_rows_count} with {unique_athletes_count} affected athletes")


In [ ]:
df_missing_both.head()

In [ ]:
# do they have other information: gender & country

# filter the rows where person_event_group is None
df_missing_both_none_group = df_missing_both[df_missing_both['person_event_group'].isna()]

# for both
with_both = df_missing_both_none_group.dropna(subset=['gender', 'person_flag'])

# for only gender
with_gender_only = df_missing_both_none_group[df_missing_both_none_group['gender'].notna() & df_missing_both_none_group['person_flag'].isna()]

# for only person_flag as country
with_country_only = df_missing_both_none_group[df_missing_both_none_group['person_flag'].notna() & df_missing_both_none_group['gender'].isna()]


# print reults
print(f"Number of 'None' rows but information on both 'gender' and 'country': {with_both.shape[0]}")
print(f"Number of 'None' rows but information on at least 'gender': {with_gender_only.shape[0]}")
print(f"Number of 'None' rows but information on at least 'country': {with_country_only.shape[0]}")


Let's get an overview how many events and participants lay behind these groups

In [ ]:
# group by event_link and aggregate the list of category names and number of unique person links
grouped = df_missing_both.groupby('event_link').agg(
    unique_groups=('person_event_group', 'unique'),
    unique_person_links=('person_link', 'nunique')
)

# sort ascending by number of unique athletes per event
grouped_sorted = grouped.sort_values(by='unique_person_links', ascending=False)

# iterate through the rows 
print(f"The {df_missing_both['event_link'].nunique():,} events with a total of {len(df_missing_both)} participations, "
      f"\ntheir corresponding group names and unique person counts " 
      f"out of {df_missing_both['person_link'].nunique():,} athletes are: \n")
for event, row in grouped_sorted.iterrows():
    # Filter out None values from unique_groups and convert the rest to strings
    valid_groups = [str(group) for group in row['unique_groups'] if group is not None]
    print(f"Event: {event}\nGroups: {', '.join(valid_groups)}\nUnique person links: {row['unique_person_links']}\n")


In [ ]:
# do they have other information: gender & country

# for both
with_both = df_missing_both.dropna(subset=['gender', 'person_flag'])

# for only gender
with_gender_only = df_missing_both[df_missing_both['gender'].notna() & df_missing_both['person_flag'].isna()]

# for only person_flag as country
with_country_only = df_missing_both[df_missing_both['person_flag'].notna() & df_missing_both['gender'].isna()]


# print reults
print(f"Out of {len(df_missing_both):,} total rows")
print(f"Number of rows but information on both 'gender' and 'country': {with_both.shape[0]:,}")
print(f"Number of rows but information on at least 'gender': {with_gender_only.shape[0]}")
print(f"Number of rows but information on at least 'country': {with_country_only.shape[0]}")


So they all have information for gender and country and I'll keep them without age-information by building an entry 'no age'

## 2. new_age_class : new system


In [ ]:
print(f"The number of entries in age_group is {df_stats['age_group'].nunique()}")
list(df_stats['age_group'].unique())

Some of the agegroups will pose some problems as they aren't unambiguously but I group them by myself

In [ ]:
# building lists per new_age_class - after checking in the next cell, for counts, 
# I integrate some ambigous groups to a class - f.e. 30-39 to list_35-44


list_U20 = [
    'U20', 'U19', '19', '18-19', '18-20','17', '17-18', '17-19', 
    '17-20',  '16', '16-17', '16-18', '16-19',  '15', '15-16', '15-17', '15-18', '15-19',  '14-15', '14-16', 
    '14-17', '14-19', '14-20',  '12-14', '12-15', '12-18', '10-14', 
    '10-16', '10-18', '01-19'
]

list_20_34 = [
    '18-22', '18-23', '18-24', '18-25', '18-29','18-30', '18-34',
    '20', '20-22', '20-24', '20-25', '20-29', '20-34', '20-39', '21', '21-30', '21-39', 
    '22', '23', '23-29', '23-34', '23-39', '24', '24-29', '24-34', '24-39', '25', 
    '25-29', '25-30', '25-31', '25-32', '25-33', '25-34', '25-35', '25-36', '25-38', 
    '25-39',  '30', '30+','30-34', '30-35', '30-36', '30-37'
]

list_35_44 = [
    '30-39', '35', '35+','35-39', '35-40', '35-41', '35-42', '35-43', '35-44', '35-45', '35-46', 
    '35-47' 
]

list_45_59 = [
    '40', '40+','40-44', '40-45', '40-46', '40-47', '40-48', '40-49', '40-50', '40-51', 
    '40-52', '40-53', '40-54', '40-55', '40-56', '40-57', '40-58', '40-59', '40-60', 
    '40-62', '40-63',  '41-50', '44', '45', '45+', 
    '45-49', '45-50', '45-51', '45-53', '45-54', '45-55', '45-57', '45-59', '50', '50+',
    '50-53', '50-54', '50-55', '50-56', '50-57', '50-58', '50-59', '50-60', 
    '50-61', '50-62', '55', '55-59', '55-60', 
]

list_60_74 = [
     '60', 
    '60+','60-64', '60-65', '60-66', '60-67', '60-69', '60-70', '60-74', '65', '65+', '65-69', '65-70', '65-74'
    '70', '70+', '70-74', 
    '70-75',
]

list_75_plus = [
    '70-79', '70-99', '75', '75+', '75-79', '75-80', '75-99', '80', '80+', 
    '80-84', '80-99', '85-89', '90+', '90-94', '90-99'
]



In [ ]:
# problematic age groups that have no clear position in the new categories

list_ambiguous = ['18-39', '18-44', '18-49', '18-50', '17-39', '17-24', '16-22', '16-24', '15-29', '14-24', '00+','16-29',
                 '25-40', '25-41', '30-40', '30-41', '30-42', '30-43', '30-44', '30-49',
                 '35-48', '35-49', '35-50', '35-52', '35-53', '35-54', '35-56', '35-59', '35-60', '35-61', '35-62', '35-63', 
                 '35-64', '35-65', '35-66',
                 '40-64', '40-65', '40-68', '40-70', 
                 '50-64', '55+',  '55-64', '55-66', 
                 '65-59', '65-99'
                 ]

# filtered dataframe by ambiguous age_class
df_filtered = df_stats[df_stats['age_group'].isin(list_ambiguous)]

# total number of participations (=rows)
total_rows = len(df_filtered)

# total number of unique athletes
unique_athletes = df_filtered['person_link'].nunique()

# number of participation per age_group
age_group_counts = df_filtered['age_group'].value_counts()

# print results
print(f"The total number of participations is {total_rows} and the number of individual athletes is {unique_athletes}\n")
print(age_group_counts.head())

I will integrate the list_ambiguous to 'no age', as it's about 5117 participation

In [ ]:
list_no_age = [None, '18-39', '18-44', '18-49', '18-50', '17-39', '17-24', '16-22', '16-24', '15-29', '14-24', '00+','16-29',
                 '25-40', '25-41', '30-40', '30-41', '30-42', '30-43', '30-44', '30-49',
                 '35-48', '35-49', '35-50', '35-52', '35-53', '35-54', '35-56', '35-59', '35-60', '35-61', '35-62', '35-63', 
                 '35-64', '35-65', '35-66',
                 '40-64', '40-65', '40-68', '40-70', 
                 '50-64', '55+',  '55-64', '55-66', 
                 '65-59', '65-99'
                 ]

In [ ]:
def categorize_age(age):
    if age in list_U20:
        return 'U20'
    elif age in list_20_34:
        return '20-34'
    elif age in list_35_44:
        return '35-44'
    elif age in list_45_59:
        return '45-59'
    elif age in list_60_74:
        return '60-74'
    elif age in list_75_plus:
        return '75+'
    else:
        return 'no age'

df_stats['new_age_class'] = df_stats['age_group'].apply(categorize_age)


# prep: gender for radial line plot

In [ ]:
# group by event_link to get the total participants per event = basis for gender % for vis in tableau
df_count = df_stats.groupby(['event_link', 'gender', 'year']).size().reset_index(name='total_count')

# bring to df_stats
df_stats = pd.merge(df_stats, df_count, on=['event_link', 'gender', 'year'], how='left')

# check result
df_stats.head()


In [ ]:
list(df_stats.columns)

# mart: before uploading to database

To get a leaner dataframe in tableau, I drop the unneeded columns

In [ ]:
# drop the helping columns of prep staging
drop_columns = ['age_group', 'is_special', 'is_multiple_categories', 'gender_letter']
df_stats_clean = df_stats.drop(columns=[col for col in drop_columns if col in df_stats.columns])

# change to a more logical order
desired_order = ['event_link', 'total_count', 'year', 'distance', 'person_link', 
                 'gender', 'special_category', 'new_age_class', 
                 'person_country', 'person_event_group']

df_stats_clean = df_stats_clean[[col for col in desired_order if col in df_stats.columns]]

df_stats_clean.head()

------------------------------

# SQAlchemy

In [ ]:
from sqlalchemy import create_engine, types
from sqlalchemy import text # to be able to pass string
from sqlalchemy.exc import SQLAlchemyError
from dotenv import dotenv_values # to load the data from .env file

preparing connection

In [ ]:
#load the data from .env file
config = dotenv_values()

# define variables for the login
pg_user = config['AZURE_USER'] 
pg_host = config['AZURE_HOST']
pg_port = config['AZURE_PORT']
pg_db = config['AZURE_DB']
pg_schema = config['AZURE_SCHEMA']
pg_pass = config['AZURE_PASS']

# build the URL
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

# create the engine
engine = create_engine(url, echo=False)
engine.url

# build the search path
my_schema = pg_schema 
with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {my_schema};'))

testing the connection

In [ ]:
def test_db_connection(engine):
    try:
        connection = engine.connect() # including 'connection' as variable to close the connection
        print("Connection successful!")
        connection.close() # closing the connection
        return True
    except SQLAlchemyError as e:
        print(f"Connection failed: {e}")
        return False

# Test the connection
test_db_connection(engine)

load DataFrames to Database

## stats

In [ ]:
df_stats_clean.to_sql('df_mart_Triathlon', con=engine, schema='public', if_exists='replace', index=False)

In [ ]:
from sqlalchemy.orm import sessionmaker

# Session for transaction controll with sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

# rollback if there is still a connection 
session.rollback()
session.close()

# end the session properly
engine.dispose()
engine = create_engine(url, echo=False)


In [ ]:
# to make sure there are no problems due to an open connection, reset engine before uploading df_stats
# isolate to autocommit = make sure that the connection is closing

#engine.dispose()
#engine = create_engine(url, isolation_level="AUTOCOMMIT", echo=False)


# as this dataset is bigger, chunk it into 1,000 blocks and set method to 'multi'

# df_stats.to_sql('df_tri_stats', con=engine, schema='public', if_exists='replace', index=False, chunksize=1000, method='multi')